<a href="https://colab.research.google.com/github/slatej1/Everside/blob/main/Historic_Everside_Deal_Cloud_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import openpyxl

In [4]:

# UPDATE INPUT FILE NAME
# Read the data from the source file
source_file_path = 'Fund III DealCloud Model - Q4 2023 vDRAFT (03.24.24).xlsx'
data = pd.read_excel(source_file_path, sheet_name='Fund III (Q4 2023)', skiprows = 2)  # Adjust sheet_name as needed

print(data.columns)

Index(['Unnamed: 0', 'Company Name', 'Date of Investment', 'Industry',
       'Geography', 'Debt (Cost)', 'Equity (Cost)', 'Total (Cost)',
       'Everside % Fund', '% Fund Concentration', 'Everside $ Expsoure',
       'Everside % Expsoure', 'Deal Contribution to Cost',
       'Deal Contribution to FMV', 'Debt (Cost).1', 'Debt Realized Value',
       'Debt (FMV)', 'Equity (Cost).1', 'Equity Realized Value',
       'Equity (FMV)', 'Cash ', 'PIK', 'Total Coupon', 'Cash', 'PIK.1',
       'Equity Ownership (FD)', 'Senior Debt', 'Sr. Debt Turns', 'Total Debt',
       'Total Debt Turns', 'Liquidity (Cash + Available Liquidity)',
       'FCCR (Covenants)', 'Enterprise Value', 'EV Multiple', 'Sales',
       'EBITDA', 'Capex', 'EBITDA - Capex', 'Senior Debt.1',
       'Sr. Debt Turns.1', 'Total Debt.1', 'Total Debt Turns.1',
       'Liquidity (Cash + Available Liquidity).1', 'FCCR (Current)',
       'Enterprise Value.1', 'EV Multiple.1', 'Sales.1', 'EBITDA.1', 'Capex.1',
       'EBITDA - Capex.

In [5]:
template_file_path = 'deal_cloud_dataset_template_for_python.xlsx'
workbook = openpyxl.load_workbook(template_file_path)
worksheet = workbook.active
# UPDATE OUTPUT FILE NAME
output_file_path = 'Fund III 2023 Q4.xlsx'

In [9]:
column_mapping = {
    'Company Name': 'E',
    'Geography': 'K',
    'Industry': 'J',
    'Date of Investment': 'I',
    'Debt (Cost)': 'Q',
    'Equity (Cost)': 'R',
    'Everside $ Expsoure': 'W', #Everside $Exposure
    'Everside % Expsoure': 'X', #Percent exposure
    'Debt (Cost).1': 'AC',
    'Debt (FMV)': 'AE',
    'Equity (Cost).1': 'AG',
    'Equity (FMV)': 'AI',
    'Cash ': 'AN',
    'PIK': 'AO',
    'Total Coupon': 'AP',
    'Cash': 'AS',
    'PIK.1': 'AT',
    'Equity Ownership (FD)': 'AU',
    'Senior Debt': 'AV',
    'Total Debt': 'AX',
    'Liquidity (Cash + Available Liquidity)': 'AZ',
    'FCCR (Covenants)': 'BA',
    'Enterprise Value': 'BB',
    'Sales': 'BD',
    'EBITDA': 'BE',
    'Capex': 'BF',
    'Senior Debt.1': 'BH',
    'Total Debt.1': 'BJ',
    'Liquidity (Cash + Available Liquidity).1': 'BO',
    'FCCR (Current)': 'BP',
    'Enterprise Value.1': 'BQ',
    'Sales.1': 'BS',
    'EBITDA.1': 'BT',
    'Capex.1': 'BU',
}

In [10]:
def col_letter_to_num(letter):
    num = 0
    for char in letter:
        num = num * 26 + (ord(char.upper()) - ord('A') + 1)
    return num

In [11]:
start_row = 2

# Set the first three columns to "Pipeline"
for i in range(len(data)):
    for col_num in range(1, 2):
        worksheet.cell(row=start_row + i, column=col_num, value="Fund III")

# for i in range(len(data)):
  #   for col_num in range(2, 4):
    #     worksheet.cell(row=start_row + i, column=col_num, value="Directs")

# for i in range(len(data)):
  #   for col_num in range(4, 5):
    #     worksheet.cell(row=start_row + i, column=col_num, value="12/31/2021")

# Write data to the template based on the column mapping
for i, row in data.iterrows():
    for source_col, target_cols in column_mapping.items():
        if isinstance(target_cols, list):
            # Handle mapping a single source column to multiple target columns
            for target_col in target_cols:
                value = row[source_col]
                col_num = col_letter_to_num(target_col)  # Convert column letter to number
                worksheet.cell(row=start_row + i, column=col_num, value=value)
        elif isinstance(target_cols, dict) and 'sources' in target_cols:
            # Handle combined or subtracted columns
            sources = target_cols['sources']
            target_col_letter = target_cols['target']
            if source_col.startswith('Subtracted'):
                # Subtract the sum of specified source columns
                value = row[sources[0]] - sum(row[source] for source in sources[1:])
            else:
                # Combine specified columns
                value = sum(row[source] for source in sources)
            col_num = col_letter_to_num(target_col_letter)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)
        else:
            # Single column mapping
            value = row[source_col]
            col_num = col_letter_to_num(target_cols)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)

In [12]:
# Save the updated template
workbook.save(output_file_path)